In [ ]:
import cudf
import cupy as cp
import pyarrow.parquet as pq
import pandas as pd

# 시각화 라이브러리
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

print(f"cuDF version: {cudf.__version__}")

In [ ]:
def optimize_memory(df):
    """ 데이터프레임의 메모리 사용량을 최적화하는 함수 """
    int_cols = df.select_dtypes(include=['int64', 'int32']).columns
    for col in int_cols:
        max_val = df[col].max()
        min_val = df[col].min()
        if max_val < 127 and min_val > -128:
            df[col] = df[col].astype('int8')
        elif max_val < 32767 and min_val > -32768:
            df[col] = df[col].astype('int16')
        elif max_val < 2147483647 and min_val > -2147483648:
            df[col] = df[col].astype('int32')
    float_cols = df.select_dtypes(include=['float64']).columns
    for col in float_cols:
        df[col] = df[col].astype('float32')
    return df

# --- PyArrow를 이용한 데이터 분할 로드 실행 ---
TRAIN_PATH = 'data/train.parquet'

# 1. PyArrow를 사용하여 CPU에서 Parquet 파일을 엽니다.
parquet_file = pq.ParquetFile(TRAIN_PATH)
batch_size = 1_000_000  # 한 번에 읽을 행의 수 (CPU RAM 및 GPU VRAM에 따라 조절)
print(f"파일을 {batch_size:,} 행 단위의 작은 조각으로 나누어 읽습니다.")

# 2. 파일의 각 조각을 순회하며 읽고, GPU로 옮긴 후 최적화합니다.
optimized_chunks = []
for i, batch in enumerate(parquet_file.iter_batches(batch_size=batch_size)):
    print(f"  - {i+1}번째 조각 처리 중...")
    # PyArrow Batch -> Pandas DataFrame으로 변환 (CPU)
    pdf = batch.to_pandas()
    # Pandas DataFrame -> cuDF DataFrame으로 변환 (CPU -> GPU 전송)
    temp_df = cudf.from_pandas(pdf)
    
    # 메모리 최적화 바로 적용
    optimized_df = optimize_memory(temp_df)
    optimized_chunks.append(optimized_df)
    
    # 메모리 확보
    del pdf, temp_df, optimized_df
    cp.get_default_memory_pool().free_all_blocks()

# 3. 최적화된 모든 조각들을 하나로 합치기
print("\n최적화된 조각들을 하나로 병합합니다...")
train_df = cudf.concat(optimized_chunks, ignore_index=True)
del optimized_chunks
cp.get_default_memory_pool().free_all_blocks()

print("\n데이터 로드 및 최적화 완료! ✨")
print(f"Final Optimized Memory Usage: {train_df.memory_usage(deep=True).sum() / 1e9:.2f} GB")

## 2. 데이터 기본 정보 확인 (Basic Information)
이제부터는 모든 작업을 `train_df` 라는 일반 `cudf` 데이터프레임으로 수행합니다.

In [ ]:
print(f"Train 데이터 형태: {train_df.shape}")
print(f"샘플 수: {train_df.shape[0]:,} 개")
print(f"컬럼 수: {train_df.shape[1]} 개")
train_df.head()

## 3. 타겟 변수(`clicked`) 분석

In [ ]:
clicked_counts = train_df['clicked'].value_counts().to_pandas()
total_ctr = (train_df['clicked'].mean() * 100).item()

fig = px.bar(clicked_counts, 
             x=clicked_counts.index, 
             y=clicked_counts.values, 
             labels={'x': 'Clicked', 'y': 'Count'},
             title=f'클릭 여부 분포 (전체 CTR: {total_ctr:.4f}%)',
             text_auto=True)
fig.show()

## 4. 주요 피처 분석 (Key Feature Analysis)

In [ ]:
main_features = ['gender', 'age_group', 'day_of_week', 'hour']

def plot_ctr_by_feature(df, feature):
    grouped = df.groupby(feature)['clicked'].agg(['count', 'sum']).to_pandas().reset_index()
    grouped['ctr'] = (grouped['sum'] / grouped['count']) * 100
    grouped = grouped.sort_values('ctr', ascending=False)
    
    fig = px.bar(grouped, 
                 x=feature, 
                 y='ctr', 
                 color='count',
                 color_continuous_scale=px.colors.sequential.Viridis,
                 title=f'{feature}에 따른 CTR',
                 labels={'ctr': 'CTR (%)', 'count': '노출 수'},
                 text_auto='.4f')
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    fig.show()

for feature in main_features:
    plot_ctr_by_feature(train_df, feature)

## 5. 익명화 피처 탐색 (Anonymous Feature Exploration)

In [ ]:
def analyze_high_cardinality_feature(df, feature):
    print(f"--- {feature} 피처 분석 ---")
    num_unique = df[feature].nunique()
    print(f"고유 값 개수: {num_unique}")
    
    grouped = df.groupby(feature)['clicked'].agg(['count', 'sum']).to_pandas().reset_index()
    grouped['ctr'] = (grouped['sum'] / grouped['count']) * 100
    
    # 노출 수 기준 상위 10개
    top_10_by_count = grouped.sort_values('count', ascending=False).head(10)
    fig1 = px.bar(top_10_by_count, x=feature, y='count', title=f'{feature}별 노출 수 TOP 10', text_auto=True)
    fig1.update_layout(xaxis_type='category')
    fig1.show()

    # CTR 기준 상위 10개 (단, 노출이 최소 1000번 이상인 경우만)
    top_10_by_ctr = grouped[grouped['count'] >= 1000].sort_values('ctr', ascending=False).head(10)
    fig2 = px.bar(top_10_by_ctr, x=feature, y='ctr', title=f'{feature}별 CTR TOP 10', text_auto='.4f')
    fig2.update_layout(xaxis_type='category')
    fig2.show()

analyze_high_cardinality_feature(train_df, 'inventory_id')
analyze_high_cardinality_feature(train_df, 'l_feat_14')